In [37]:
import urllib.request
import json
import collections
import sqlite3
import os
from datetime import datetime
from pymystem3 import Mystem
import os
m = Mystem()
import numpy as np

In [37]:
def getFiles():
    
    # Функция извлекает тексты из БД и записывает в файлы
    
    conn = sqlite3.connect('Vedaworld Posts.db')
    c = conn.cursor()
    for row in c.execute("SELECT text, id FROM vedas"):
        text = row[0]
        post_id = row[1]
        text = text.replace('\\n', '\n')
        path = "." + os.sep + "Vedaworld" + os.sep + "plain"
        if not os.path.exists(path):
            os.makedirs(path)
        file = "%s\\%s.txt" % (path, str(post_id))
        with open(file, 'w', encoding='utf-8') as t:
            t.write(text)
    conn.commit()
    conn.close()

In [9]:
def getComments():
    
    # Функция создает таблицу, где id - номер поста, а в строках хранятся \
    # комментарии и информация про автора.
    
    conn = sqlite3.connect('Vedaworld Posts.db')
    c = conn.cursor()
    c.execute("CREATE TABLE IF NOT EXISTS comments (id, date, \
    city, age, sex, university, comment_id, text, lemmas, url)")
    c.execute("SELECT id FROM comments")
    for row in c.execute("SELECT id, com_number FROM vedas"):
        post_id = row[0]
        com_number = row[1]
        if int(com_number) > 100:
            offset = ((int(234)//100)+1)*100
            offset = [i for i in range(0, offset, 100)]
        else:
            offset = [0]
            
            # Обход постов по изветным id и сохранение комментариев:
            
        for off in offset:
            req = urllib.request.Request('https://api.vk.com/method/\
            wall.getComments?owner_id=-107513904&offset=%s&extended=1&post_id=%s&\
            count=100&v=5.92&access_token=%s' % (off, post_id, token)) 
            response = urllib.request.urlopen(req) 
            result = response.read().decode('utf-8')
            data = json.loads(result)
            try:
                for i in range(data['response']['current_level_count']):
                    date, city, age, sex, university, comment_id, \
                    text, lemmas, url = '', '', '', '', '', '', '', '', ''
                    try:
                        user_id = data['response']['profiles'][i]['id']
                    except:
                        pass
                    try:
                        sex = data['response']['profiles'][i]['sex']
                    except:
                        pass
                    try:
                        text = data['response']['items'][i]['text']
                    except:
                        pass
                    try:
                        unixtime = data['response']['items'][i]['date']
                        date = datetime.fromtimestamp(unixtime)
                    except:
                         pass

                    # Обход страниц пользователей    
                        
                    comment_id = i
                    req = urllib.request.Request('https://api.vk.com/method/\
                    users.get?user_ids=%s&fields=sex,bdate,universities,home_town&v=5.92\
                    &access_token=%s' % (user_id, token)) 
                    response = urllib.request.urlopen(req)
                    result = response.read().decode('utf-8')
                    data_person = json.loads(result)
                    if data_person['response'][0]['is_closed'] == False:
                        try:
                            bdate = data_person['response'][0]['bdate']
                            age = date.year - int(bdate.split('.')[2])
                        except:
                            pass
                        try:
                            university = data_person['response'][0]['universities'][0]['name']
                        except:
                            pass
                        try:
                            city = data_person['response'][0]['home_town']
                        except:
                            pass
                    c = conn.cursor()
                    c.execute("INSERT INTO comments VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",\
                          (post_id, date, city, age, sex, university,\
                           comment_id, text, lemmas, url))
                    conn.commit()
            except:
                pass
    conn.close()

In [5]:
SYMBS = "1234567890,—[]↑№!\"\'«»?.,;:|/\+*{}<>@#$%-^& )("

def clean(text):
    
    # Функция для очистки текста от лишних символов
    
    words = []
    clean_text = text.strip().lower().split()
    for word in clean_text:
        word_new = word.strip(SYMBS)
        if word_new != '' and word_new != '–':
            words.append(word_new)
    return words

def clean_lemmas(words):
    
    # Функция для лемматизации из списка слов
    
    text = ' '.join(words)
    lemmas = m.lemmatize(text)
    return lemmas

In [11]:
def lemmatizeFiles():
    
    # Ламматизация текстов постов
    
    for i in range(1, 200):
        path = "." + os.sep + "Vedaworld" + os.sep + "plain"
        file = "%s\%s.txt" % (path, str(i))
        with open(file, 'r', encoding='utf-8') as f:
            text = f.read()
        text = clean(text)
        text = clean_lemmas(text)
        path = "." + os.sep + "Vedaworld" + os.sep + "lemmatized"
        if not os.path.exists(path):
            os.makedirs(path)
        file = "%s\%s.txt" % (path, str(i))    
        with open(file, 'w', encoding='utf-8') as t:
            t.write(''.join(text))

In [92]:
def lemmatizeComments():
    
    # Лемматизирует тексты комментариев. Вылетает ошибка при лемматизации смайлов.
    
    path = "." + os.sep + "Vedaworld" + os.sep + "plain_comments"
    for file in os.listdir(path):
        print(file)
        filename = "%s\%s" % (path, file)
        with open(filename, 'r', encoding='utf-8') as f:
            text = f.readlines()
        new_text = []
        #try:
        if len(text) > 1:
            for i in range(1, len(text), 3):
                print(text[i])
 
                line = clean(text[i])
                line = clean_lemmas(line)
                com_len = len(line)

                line = text[i]
                line = ''.join(line)
                new_text.append(line)
        else:
            line = clean(text[0])
            line = clean_lemmas(line)
            line = text[0]
            new_text.append(line)

        new_path = "." + os.sep + "Vedaworld" + os.sep + "lemmatized_comments"
        if not os.path.exists(new_path):
            os.makedirs(new_path)
        filename = "%s\%s" % (new_path, file)    
        with open(filename, 'w', encoding='utf-8') as t:
            t.write('\n\n'.join(new_text))
        

In [41]:
def commentsFiles():
    
    # Сохранение комментариев в файлы из БД
    
    conn = sqlite3.connect('Vedaworld Posts.db')
    c = conn.cursor()
    for row in c.execute("SELECT text, comment_order, id FROM comments"):
        text = row[0]
        order = row[1]
        post_id = row[2]
        text = text.replace('\\n', '\n')
        path = "." + os.sep + "Vedaworld" + os.sep + "plain_comments"
        file = "%s\\%s.txt" % (path, str(post_id))
        if not os.path.exists(path):
            os.makedirs(path)
        with open(file, 'a', encoding='utf-8') as t:
            t.write('{}\n{}\n\n'.format(str(order), text))
    conn.commit()
    conn.close()

In [39]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [56]:
def veryClean(folder):
    
    # Убираем стоп-слова и выдаем список слов из нужной папки
    
    path = "." + os.sep + "Vedaworld" + os.sep + folder
    words_new = []
    for file in os.listdir(path):
        filename = "%s\%s" % (path, file)
        with open(filename, 'r', encoding='utf-8') as f:
            text = f.read()
            with open('rus_stopwords.txt', 'r', encoding='utf-8') as f:
                stop = [i.strip('\n') for i in f.readlines()]
            text = clean(text)
            for word in text:
                if word not in stop:
                    words_new.append(word)
    return words_new

In [68]:
def freq():
    
    # График наиболее частотных слов без лемматизации:
    
    words = veryClean('plain_comments')
    words_plain = veryClean('plain')
    words_plain.extend(words)

    counter = collections.Counter(words_plain)
    
    y = [counter.most_common(20)[i][1] for i in range(20)]
    x = [counter.most_common(20)[i][0] for i in range(20)]
    colors = ['red', 'yellow', 'green', 'pink', 'blue', \
              'purple', 'black', 'orange', 'violet', 'brown']
    plt.title('Самые частые слова без лемматизации')
    plt.xticks(rotation = 90)
    plt.xlabel('Слова')
    plt.ylabel('Количество')
    plt.bar(x, y, color=colors, edgecolor='black')
    plt.savefig('Most_common_20_plain.png')
    
    # С лемматизацией:
    
    lemmas = veryClean('lemmatized')
    counter = collections.Counter(lemmas)
    y = [counter.most_common(20)[i][1] for i in range(20)]
    x = [counter.most_common(20)[i][0] for i in range(20)]
    colors = ['red', 'yellow', 'green', 'pink', 'blue', \
              'purple', 'black', 'orange', 'violet', 'brown']
    plt.title('Самые частые слова')
    plt.xticks(rotation = 90)
    plt.xlabel('Слова')
    plt.ylabel('Количество')
    plt.bar(x, y, color=colors, edgecolor='black')
    plt.savefig('Most_common_20_lemmatized.png')
    

In [58]:
def cloud():
    
    # Создает облако слов по лемматизированным постам
    
    words = veryClean('lemmatized')
    text_cloud = ' '.join(words) 
    from wordcloud import WordCloud
    comment_mask = np.array(Image.open("comment.jpg"))
    
    cloud = WordCloud(background_color="white", max_words=2000, mask=comment_mask)
    
    cloud.generate(text_cloud)
    
    plt.imshow(cloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()
    cloud.to_file("post_cloud.png")

In [47]:
def main():
    getPosts()
    getFiles()
    getComments()
    commentsFiles()
    lemmatizeFiles()
    freq()
    cloud()

In [ ]:
if __name__ == '__main__':
    main()